# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

* Initial state
* Actions
* Transition model
* Goal state

In [1]:
# Your code/answer goes here.

|||
|:-----------------|------|
|**Initial State**    | <li> &nbsp;&nbsp; Defined by the location of an agent's piece ('x', 'o') within the bottom row </li> |
|**Actions**          | <li> &nbsp;&nbsp; { Left, Right } </li> |
|**Transition Model** | <li> &nbsp;&nbsp; The agent may choose any column _C_, out of the 7 total, <br/> &nbsp;&nbsp;&nbsp; that is not yet full:  __7 - C__ </li> |
|**Goal State**       | <li> &nbsp;&nbsp; Put n 'x' or 'o' pieces on a 6 × 7 board having 4 (or more) <br/> &nbsp;&nbsp;&nbsp; of the same piece type ('x' or 'o') line up together in either <br/> &nbsp;&nbsp;&nbsp; the same row, column, or diagonally. This line must <br/> &nbsp;&nbsp;&nbsp; not have any gaps in between </li> |
|**Path Cost**        | <li> &nbsp;&nbsp; 1 per move </li> |

How big is the search space?

In [2]:
# Your code/ answer goes here.

- Our board is 6 x 7, thus the total number of locations on the board is <u>**42**</u>.
- Each location (row,column) within the board has <u>**3**</u> Possible states : 'x', 'o', or empty 

So that then gives us an `upper bound` of  $3^{42}$ , which isn't necessarily true.

This calculation implies that an agent could place a piece within any row at any given point, which is false.  
An example is if an agent wanted to place their first piece in the middle row. Due to gravity, the piece would  
drop down to the bottom row.

The best `lower bound` on the number of possible positions has been calculated by a computer program to be around _$1.6 * 10^{13}$_  [1].
    

## Task 2: Game Environment and Random Agent [2 point]

Use a numpy character array as the board.

In [3]:
import numpy as np

def empty_board(shape=(6, 7)):
    return np.full(shape=shape, fill_value=' ')

print(empty_board())

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']]


Instead of colors for the players use 'x' and 'o' to represent the players. Make sure that your agent functions all have the from: `agent_type(board, player = 'x')`, where board is the current board position and player is the player whose next move it is and who the agent should play.

Implement the board and helper functions for:

* The transition model (result).
* The utility function.
* Check for terminal states.
* A check for available actions.
* A function to visualize the board.

Make sure that all these functions work with boards of different sizes.

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected? 

In [4]:
# Your code/ answer goes here.

In [18]:
import math

board = empty_board()

# Add move to the board -------------------------------------------------------------------------- 
def result(state, player, action):
    state = state.copy()
    
    for row in range(len(state[0])-1,0,-1):
        if state[row - 1][action] == ' ':
            state[row - 1][action] = player
            break
            
        if row - 1 == -1 and row == 0 and state[row][action] == ' ': 
            state[row - 1][action] = player
            break
            
        if state[row - 1][action] != ' ' and (row - 1 > 0): continue
                
    #print("--------------- State ---------------\n", state)
    #print("--------------- Player ---------------\n", player)
    #print("--------------- Action ---------------\n", action)
  
    return state

# Produce the Belief State* after the given action for a player. ---------------------------------
# *Belief State: The set of boards w/ the action & all possible reactions of an opponent.
def results(board, action, player = 'x'):
    if player == 'x': other = 'o'
    else: other = 'x'
    
    board = board.copy()
    
    # player's move
    valid_move = False
    while not valid_move:
        for row in range(len(board[0])-1,0,-1):
            if board[row - 1][action] == ' ':
                board[row - 1][action] = player
                valid_move = True
                break
            
            if row - 1 == -1 and row == 0 and board[row][action] == ' ': 
                board[row - 1][action] = player
                valid_move = True
                break
            
            if board[row - 1][action] != ' ' and (row - 1 > 0):
                continue
            
    # opponent reacts
    r = list()
    o_actions = actions(board)
    
    # board is full
    if len(o_actions) < 1 : return [board]
    
    for o_a in o_actions:
        s = board.copy()
        s[o_a] = other
        r.append(s)    
    
    return r

# Check the board & return : 'x', 'o', 'd' (draw), or n (next move)
def check_win(board):
    num_rows, num_cols = board.shape
    
    # Vertical Check
    for row in range(0, num_rows):
        for col in range(0, num_cols):
            if num_rows - row < 3: break
                
            if num_rows - row >= 4:
                if board[row][col] != ' ':
                    if board[row][col] == board[row + 1][col]:
                        if board[row + 1][col] == board[row + 2][col]:
                            if board[row + 2][col] == board[row + 3][col]:
                                return board[row + 3][col]
    
    # Horizontal Check 
    half_r = math.ceil((num_rows / 2) - 1)
    half_c = math.ceil((num_cols / 2))
    for rows in range(0, num_rows):
        for cols in range(0, num_cols - 1):
            if num_cols - cols < 4: break
            if cols == num_cols: break
                
            if cols < num_cols:
                if board[rows][cols] != ' ':
                    if board[rows][cols] == board[rows][cols + 1]:
                        if board[rows][cols + 1] == board[rows][cols + 2]:
                            if board[rows][cols + 2] == board[rows][cols + 3]:
                                return board[rows][cols]
    
    # Diagonal Check : BottomLeft to TopRight                   
    for row in range(num_rows - 1, 2, -1):
        for col in range(half_c):
            if board[row][col] != ' ':
                if board[row][col] == board[row - 1][col + 1]:
                    if board[row -1][col + 1] == board[row - 2][col + 2]:
                        if board[row - 2][col + 2] == board[row - 3][col + 3]: 
                            return board[row][col]
                        
    # Diagonal Check : TopLeft to BottomRight                    
    for row in range(half_r + 1):
        for col in range(half_c):
            if board[row][col] != ' ':
                if board[row][col] == board[row + 1][col + 1]:
                    if board[row + 1][col + 1] == board[row + 2][col + 2]:
                        if board[row + 2][col + 2] == board[row + 3][col + 3]:
                            return board[row][col]
                        
    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'
                    
# Returns win/loss (terminal) or false (non-terminal) --------------------------------------------
def is_terminal(state, player = 'x', draw_is_win = True):
    if player == 'x': other = 'o'
    else: other = 'x'
    
    goal = check_win(state)
    if goal == str(player): return 'win' 
    if goal == 'd': 
        if draw_is_win: return 'draw' 
        else: return None 
    if goal == other: return None  # loss is failure
    return False # continue

# Return possible actions as a vector of idx's ---------------------------------------------------
def actions(board):
    valid_columns = []
    for i in np.where(np.array(board == ' ')[0]):
        valid_columns.append(i)
    return np.array(valid_columns)[0]

# Display the board ------------------------------------------------------------------------------
def show_board(board):
    print()
    print(board)
    print()
    # print(np.array(board))

In [19]:
# Simple player that chooses a random empty square. Player is unused. ----------------------------  
def random_player(board, player = None):
    val = np.random.choice(actions(board))
    # print("Random Pick: ", val)
    return val

def switch_player(player, x, o):
    if player == 'x': return 'o', o
    else: return 'x', x
    
def is_winning(board, player):
    winning = False
    for i in [0,1,2]:
        if board[i][0] == board[i][1] == board[i][2] == player:
            winning = True
            
    for j in [0,1,2]:
        if board[0][j] == board[1][j] == board[2][j] == player:
            winning = True
            
        if board[0][0] == board[1][1] == board[2][2] == player:
            winning = True
            
        if board[0][2] == board[1][1] == board[2][0] == player:
            winning = True
            
    return winning


# 2 Random Agents, N times
def play(x, o, N = 100):
    """ x starts. x & y are agent functions that get the board as the percept and return their next action."""
    results = {'x': 0, 'o': 0, 'd': 0}
    
    for i in range(N):
        board = empty_board()
        player, fun = 'x', x
        
        while True:
            a = fun(board, player)
            board = result(board, player, a)
            
            win = check_win(board)   # returns the 'n' if the game is not done.
            if win != 'n':
                results[win] += 1
                break
            
            player, fun = switch_player(player, x, o)
            
            #print(board, "\n")
    
    return results

In [20]:
%timeit -n 1 -r 1 display(play(random_player, random_player, N = 1000))

{'x': 544, 'o': 454, 'd': 2}

3.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Each of the players result in winning about half of all games. This is something that can be expected  
when choosing a randomized action.

## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.



__Note:__ The search space for a $6 \times 7$ board is large. You can experiment with smaller boards (the smallest is $4 \times 4$) and/or changing the winning rule to connect 3 instead of 4.

In [21]:
# Your code/ answer goes here.

In [22]:
board = empty_board(shape=(4,4))

print("Win? ", check_win(board))

Win?  n


Experiment with some manually created boards (at least 5) to check if the agent spots winning opportunities.

In [23]:
# Your code/ answer goes here.

In [26]:
def minimax_ab(board):
    alpha, beta = None, None
    max_trans, max_util = None, None
    
    transitions = results(board, 'x')
    for trans, nextstate in transitions.iteritems():
        u = min_utility_ab(nextstate, 'o', alpha, beta)
        if max_u is None or u > max_u:
            max_trans = trans
            max_u = u
            
        if beta is not None and u >= beta: return trans
        
        if alpha is None or u > alpha: alpha = u
            
    return max_trans

def min_utility_ab(board, player, alpha, beta):
    if is_winning(board, player) or is_winning(board, switch_player(player)) or is_tie(board):
        return utility(board, 'x')
    else:
        transitions = results(board, player)
        min_u = None
        
        for nextstate in transitions.values():
            u = max_utility_ab(nextstate, switch_player(player), alpha, beta)
            
            if min_u is None or u < min_u: min_u = u
            
            if alpha is not None and u <= alpha: return u
            
            if beta is None or u < beta: beta = u
        
        return min_u

def max_utility_ab(board, player, alpha, beta):
    
    if is_winning(board, player) or is_winning(board, switch_player(player)) or is_tie(board):
        return utility(board, 'x')
    
    else:
        transitions = possible_transitions(board, player)
        
        max_u = None
        
        for nextstate in transitions.values():
            u = min_utility_ab(nextstate, switch_player(player), alpha, beta)
            
            if max_u is None or u > max_u: max_u = u
                
            if beta is not None and u >= beta: return u
            
            if alpha is None or u > alpha: alpha = u
                
        return max_u

In [27]:
print('----------- Board 1 -----------')
board_1 = empty_board(shape=(4,4))
board_1[3][0] = 'x'
board_1[3][1] = 'o'
board_1[2][1] = 'x'
board_1[3][2] = 'o'
board_1[2][2] = 'x'
board_1[3][3] = 'o'
board_1[1][2] = 'x'
board_1[2][3] = 'o'
board_1[2][0] = 'x'
board_1[1][3] = 'o'
show_board(board_1)
minimax_ab(board_1)

print('----------- Board 2 -----------')
board_2 = empty_board(shape=(4,4))
board_2[3][3] = 'x'
board_2[3][0] = 'o'
board_2[2][3] = 'x'
board_2[3][1] = 'o'
board_2[1][3] = 'x'
board_2[3][2] = 'o'
show_board(board_2)

print('----------- Board 3 -----------')
board_3 = empty_board(shape=(4,4))
board_3[3][0] = 'x'
board_3[2][0] = 'o'
board_3[3][1] = 'x'
board_3[2][1] = 'o'
board_3[3][2] = 'x'
board_3[2][2] = 'o'
show_board(board_3)

print('----------- Board 4 -----------')
board_4 = empty_board(shape=(4,4))
board_4[3][3] = 'x'
board_4[3][2] = 'o'
board_4[2][2] = 'x'
board_4[3][1] = 'o'
board_4[1][1] = 'x'
board_4[2][1] = 'o'
show_board(board_4)

print('----------- Board 5 -----------')
board_5 = empty_board(shape=(4,4))
board_5[3][0] = 'x'
board_5[3][1] = 'o'
board_5[2][0] = 'x'
board_5[3][3] = 'o'
board_5[1][0] = 'x'
board_5[2][3] = 'o'
show_board(board_5)

----------- Board 1 -----------

[[' ' ' ' ' ' ' ']
 [' ' ' ' 'x' 'o']
 ['x' 'x' 'x' 'o']
 ['x' 'o' 'o' 'o']]



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
# Your code/ answer goes here.

### Move ordering

Describe and implement a simple move ordering strategy. How does this strategy influence the time it takes to 
make a move?

In [ ]:
# Your code/ answer goes here.

### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [ ]:
# Your code/ answer goes here.

## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [ ]:
# Your code/ answer goes here.

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [ ]:
# Your code/ answer goes here.

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [ ]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
# Your code/ answer goes here.

### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [ ]:
# Your code/ answer goes here.

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

In [ ]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

In [ ]:
# Your code/ answer goes here.


--------------

_References_

[1]
“Connect Four,” 2010. [Online]. Available: https://web.mit.edu/sp.268/www/2010/connectFourSlides.pdf.
‌